<a href="https://www.oguzerdogan.com/">
    <img src="https://www.oguzerdogan.com/wp-content/uploads/2020/10/logo_oz.png" width="200" align="right">
</a>

<center><h1><strong>Pima Indians Diabetes Classification Project</strong></h1>
<img
src="https://cdn.britannica.com/s:700x500/42/93542-050-E2B32DAB/women-Pima-shinny-game-field-hockey.jpg">
</center>

# &#127919; Objective of Kernel

Diabetes, is a group of metabolic disorders in which there are high blood sugar levels over a prolonged period. Symptoms of high blood sugar include frequent urination, increased thirst, and increased hunger. If left untreated, diabetes can cause many complications. Acute complications can include diabetic ketoacidosis, hyperosmolar hyperglycemic state, or death. Serious long-term complications include cardiovascular disease, stroke, chronic kidney disease, foot ulcers, and damage to the eyes.

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. 

The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are **females at least 21 years old of Pima Indian heritage.**

--------------------

**Details about the dataset:**
The datasets consists of several medical predictor variables and one target variable, Outcome. Predictor variables includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.

**Pregnancies:** Number of times pregnant  
**Glucose:** Plasma glucose concentration a 2 hours in an oral glucose tolerance test  
**BloodPressure:** Diastolic blood pressure (mm Hg)  
**SkinThickness:** Triceps skin fold thickness (mm)  
**Insulin:** 2-Hour serum insulin (mu U/ml)  
**BMI:** Body mass index (weight in kg/(height in m)^2)  
**DiabetesPedigreeFunction:** Diabetes pedigree function  
**Age:** Age (years)  
**Outcome:** Class variable ( 0 - 1)

-----------

**Number of Observation Units:** 768  
**Variable Number:** 9

Result; The model with the highest score after hyper parameter optimization was LGBM with 0.90 cross validation score.

In [ ]:
%%html
<style> 
@import url('https://fonts.googleapis.com/css?family=Orbitron|Roboto');
a {color: #37c9e1; font-family: 'Roboto';} 
h1 {color: #C20E69; font-family: 'Poppins'} 
h2, h3 {color: #25B89B; font-family: 'Poppins';}
h4 {color: #818286; font-family: 'Roboto';}
                                      
</style>

# &#128217; Load Libraries

In [ ]:
# Main

import pandas as pd
import numpy as np
import warnings

# Plots

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.tools as tls
import plotly.figure_factory as ff
import itertools
plt.style.use('fivethirtyeight')
py.offline.init_notebook_mode(connected=True)

# Models & Others

from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
warnings.simplefilter(action="ignore")


# &#128214; Read Data

In [ ]:
df = pd.read_csv(r"../input/pima-indians-diabetes-database/diabetes.csv")
df.head()

## &#128270; Descriptive Statistics

In [ ]:
df.describe([0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

<div class="alert alert-block alert-info" style="margin-top: 20px">

<ul>
<li><strong>pregnancies</strong> mean &gt; median, <strong>right skewed</strong> | <strong>99% quartile value:</strong> 13, <strong>max value:</strong> 17 <strong>might be outlier</strong></li>
<li><strong>glucose</strong> mean &gt; median, <strong>right skewed</strong> | <strong>%99</strong> and <strong>max</strong> values close each other</li>
<li><strong>bloodpressure</strong> mean = median nearly, looks <strong>gaussian distribution</strong>. There is a 16 units difference between <strong>%99</strong> and <strong>max value. might be outlier</strong></li>
<li><strong>skinthickness</strong> mean &gt; median, <strong>right skewed</strong> | <strong>min value:</strong> 0 | <strong>99% quartile value:</strong> 51 <strong>max value:</strong> 99 <strong>there is outlier</strong></li>
<li><strong>insulin</strong> mean(79) &gt; median(30), <strong>right skewed</strong> | <strong>min value:</strong> 0, <strong>99% quartile value:</strong> 519 <strong>max value:</strong> 846</li>
<li><strong>BMI</strong> mean(31) =~ median(32), looks <strong>gaussian distribution</strong> | <strong>min value:</strong> 0 | <strong>99% quartile value:</strong> 50, <strong>max value:</strong> 67</li>
<li><strong>DPF</strong> mean(0.47) &gt; median (0.37) | <strong>min value:</strong> 0.078 | <strong>99% quartile value:</strong> 1.69 <strong>max value:</strong> 2.42</li>
<li><strong>Age</strong> mean(33)&gt;median(29) <strong>right skewed</strong> | <strong>min value:</strong> 21 | <strong>99% quartile value:</strong> 67 <strong>max value:</strong> 2.42</li>

</ul>

</div>

In [ ]:
# Data Shape
print("There are {} observation and {} features ".format(df.shape[0], df.shape[1]))

## &#128918; Target Variable Counts

In [ ]:
#------------COUNT-----------------------
def target_count():
    trace = go.Bar( x = df['Outcome'].value_counts().values.tolist(), 
                    y = ['healthy','diabetic' ], 
                    orientation = 'h', 
                    text=df['Outcome'].value_counts().values.tolist(), 
                    textfont=dict(size=20),
                    textposition = 'auto',
                    opacity = 0.8,marker=dict(
                    color=['#25B89B', '#C20E69'],
                    line=dict(color='#FFFFFF',width=1.5)))

    layout = dict(title =  'Count of Outcome variable')

    fig = dict(data = [trace], layout=layout)
    py.iplot(fig)
target_count()

## &#9703; Categorical & Numerical Columns

In [ ]:
def cols():
    cat_cols = [col for col in df.columns if df[col].dtypes == "O"]
    if len(cat_cols) == 0:
        print("There is not Categorical Column")
    else:
        print("Number of Categorical Column: ", len(cat_cols),"\n",cat_cols)
    
    num_cols = [col for col in df.columns if df[col].dtypes != "O"]
    if len(num_cols) == 0:
        print("There is not Numerical Column")
    print("Number of Numerical Columns: ", len(num_cols),"\n",num_cols)
cols()

# &#128270; Missing Values

<div class="alert alert-danger" role="alert">
    In this data set <b>NA</b> are filled with 0
</div>

In [ ]:
# Missing Values Table Function
def missing_values_table(dataframe):
    
    variables_with_na = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[variables_with_na].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[variables_with_na].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    dtypes = dataframe.dtypes
    dtypesna = dtypes.loc[(np.sum(dataframe.isnull()) != 0)]
    missing_df = pd.concat([n_miss, np.round(ratio, 2), dtypesna], axis=1, keys=['n_miss', 'ratio', 'type'])
    if len(missing_df)>0:
        print(missing_df)
        print("\nThere are {} columns with missing values\n".format(len(missing_df)))
    else:
        print("\nThere is no missing value") 



In [ ]:
missing_values_table(df)

In [ ]:
# This features can not be 0!
missing = ["Glucose", "BMI", "BloodPressure", "SkinThickness", "Insulin"]

for i in missing:
    df[i] = np.where(df[i] == 0, np.nan, df[i])

In [ ]:
missing_values_table(df)

## &#9850; Filling NA Values

In [ ]:
#The missing values will be filled with the median values of each variable
def median_target(variable):   
    temp = df[df[variable].notnull()]
    temp = temp[[variable, 'Outcome']].groupby(['Outcome'])[[variable]].median().reset_index()
    return temp

In [ ]:
#The values to be given for incomplete observations are given the median value of people who are not sick and the median values of people who are sick
columns = df.columns
columns = columns.drop("Outcome")
for i in columns:
    median_target(i)
    df.loc[(df['Outcome'] == 0 ) & (df[i].isnull()), i] = median_target(i)[i][0]
    df.loc[(df['Outcome'] == 1 ) & (df[i].isnull()), i] = median_target(i)[i][1]

In [ ]:
missing_values_table(df)

-----------------

# &#128202; Exploratory Data Analysis

## Histogram

In [ ]:
df.hist(bins=20,color = "#F19C1F",edgecolor='white',figsize = (15,15));

## Correlation Between Features

In [ ]:
corr_matrix = df.corr()
sns.clustermap(corr_matrix, annot = True, fmt = ".2f", cmap = "viridis", figsize=(11,11))
plt.title("Correlation Between Features")
plt.show()

-------------------
# &#9823; Base Models | LightGBM

In [ ]:
# Split X and y
y = df["Outcome"]
X = df.drop(["Outcome"], axis = 1)

In [ ]:
log_model = LogisticRegression().fit(X,y)
y_pred = log_model.predict(X)
print("Accuracy Score:", accuracy_score(y, y_pred), "\n")
print(classification_report(y,y_pred))

In [ ]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('GBM',GradientBoostingClassifier()))
models.append(('XGB', GradientBoostingClassifier()))
models.append(("LightGBM", LGBMClassifier()))

# evaluate each model in turn
results = []
names = []

for name, model in models:
    
        cv_results = cross_val_score(model, X, y, cv = 10, scoring= "accuracy")
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)

## &#128202; Base Model: LightGBM Feature Importance

In [ ]:
Importance = pd.DataFrame({'Importance':LGBMClassifier().fit(X, y).feature_importances_*100}, 
                          index = X.columns)

Importance.sort_values(by = 'Importance', 
                       axis = 0, 
                       ascending = True).plot(kind = 'barh', 
                                              color = '#25B89B', figsize=(10,6))

plt.xlabel('LightGBM Feature Importance')
plt.gca().legend_ = None

--------------------------
# &#128296; Feature Engineering


## Outliers

In [ ]:
Q1 = df.Insulin.quantile(0.25)
Q3 = df.Insulin.quantile(0.75)
IQR = Q3-Q1
lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR
df.loc[df["Insulin"] > upper,"Insulin"] = upper

In [ ]:
import seaborn as sns
sns.boxplot(x = df["Insulin"]);

## New Features

In [ ]:
# According to feature importance i'm creating new features.
df2 = df.copy()

df2["Insulin/Age"]=df2["Insulin"]/df2["Age"]
df2["BMI/Age"]=df2["BMI"]/df2["Age"]
df2["Pregnancies/Age"]=df2["Pregnancies"]/df2["Age"]
df2["Ins*Glu"]=df2["Insulin"]* df2["Glucose"]
df2.drop(["Age"],axis = 1, inplace = True)

## Edit Features

### - BMI Levels

<center><img
src="https://www.oguzerdogan.com/wp-content/uploads/2020/10/BMI.jpg">
</center>

In [ ]:
df2['New_BMI'] = pd.cut(x = df['BMI'], bins = [0,18.5, 24.9, 29.9, 100], labels = ["Underweight", 
                                                                                  "NormalWeight", 
                                                                                  "Overweight", 
                                                                                  "Obes"])

### - Blood Pressure Levels

<center><img
src="https://www.oguzerdogan.com/wp-content/uploads/2020/10/blood.jpg">
</center>



In [ ]:
df2['New_BloodPressure'] = pd.cut(x = df['BloodPressure'], bins = [0,80, 90, 120, 122], labels = ["Normal", 
                                                                                                "Hyper_St1", 
                                                                                                "Hyper_St2", 
                                                                                                "Hyper_Emer"])
#reference: American Heart Association

### - Glucose Levels

<center><img
src="https://www.oguzerdogan.com/wp-content/uploads/2020/10/glucose.jpg">
</center>

In [ ]:
df2["New_Glucose"] = pd.cut(x = df["Glucose"], bins = [0,140,200,300], labels = ["Normal",
                                                                                "Prediabetes",
                                                                                "Diabetes"])
#reference: https://emedicine.medscape.com/article/2049402-overview

### - Insulin Levels

In [ ]:
#A categorical variable creation process is performed according to the insulin value.
def set_insulin(row): 
    if row["Insulin"] >= 100 and row["Insulin"] <= 126:
        return "Normal"
    else:
        return "Abnormal"

In [ ]:
df2 = df2.assign(NewInsulinScore=df2.apply(set_insulin, axis=1))

In [ ]:
df2.head()

------------
# &#8690; One Hot Encoding

In [ ]:
def one_hot_encoder(dataframe, categorical_columns, nan_as_category=False):
    original_columns = list(dataframe.columns)
    dataframe = pd.get_dummies(dataframe, columns=categorical_columns,
                               dummy_na=nan_as_category, drop_first=True)
    new_columns = [col for col in dataframe.columns if col not in original_columns]
    return dataframe, new_columns

In [ ]:
categorical_columns = [col for col in df2.columns
                           if len(df2[col].unique()) <= 10
                      and col != "Outcome"]
categorical_columns

In [ ]:
df2, new_cols_ohe = one_hot_encoder(df2,categorical_columns)
new_cols_ohe

In [ ]:
df2.head()

----------------
# &#128270; LOF

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
lof =LocalOutlierFactor(n_neighbors= 20)
lof.fit_predict(df2)
df_scores = lof.negative_outlier_factor_
np.sort(df_scores)[0:30]

In [ ]:
th = np.sort(df_scores)[6]
th

In [ ]:
#We delete those that are higher than the threshold value
df2 = df2[df_scores > th]
df2.shape

----------
# &#9822; New Features Model | LightGBM

In [ ]:
y = df2["Outcome"]
X = df2.drop(["Outcome"], axis = 1)

In [ ]:
models = [('RF', RandomForestClassifier()),
          ('GBM',GradientBoostingClassifier()),
          ('XGBM', XGBClassifier()),
          ("LightGBM", LGBMClassifier())]

# evaluate each model in turn
results = []
names = []

for name, model in models:
    kfold = KFold(n_splits=10, random_state=123456)
    cv_results = cross_val_score(model, X, y, cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

-------------------
# &#9881;Model Tuning Steps

In [ ]:
# LGBM Tuned Model

lgbm_tuned = LGBMClassifier(colsample_bytree = 0.5, 
                            learning_rate = 0.01,
                            max_depth = 6,
                            n_estimators = 500).fit(X, y)

In [ ]:
# GBM Tuned Model

gbm_tuned = GradientBoostingClassifier(learning_rate = 0.1,
                                      max_depth = 3,
                                      n_estimators = 200,
                                      subsample = 0.8).fit(X,y)

In [ ]:
# XGBoost Tuned Model

xgb_tuned = XGBClassifier(learning_rate = 0.01,
                         max_depth = 3,
                         n_estimators =1000,
                         subsample = 1.0).fit(X,y)

In [ ]:
# evaluate each model in turn
models = [("GBM", gbm_tuned),
         ("XGBoost", xgb_tuned),
         ("LightGBM", lgbm_tuned)]
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=123456)
    cv_results = cross_val_score(model, X, y, cv=10, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

----------------------------
# &#9812;Final Model | LightGBM: 0.90

In [ ]:
feature_imp = pd.Series(lgbm_tuned.feature_importances_,
                        index=X.columns).sort_values(ascending=False)

sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('LightGBM: 0.90 Accuracy')
plt.title("Feature Importance")
plt.show()

In [ ]:
# Checking Overfitting
log_model = LogisticRegression().fit(X,y)
y_pred = log_model.predict(X)
print(accuracy_score(y, y_pred))
print(classification_report(y, y_pred))

<div class="alert alert-warning" role="alert" style="margin-top: 20px">

<h1>REPORT</h1>

<p><strong>The aim of this study</strong> was to create classification models for the diabetes data set and to predict whether a person is sick by establishing models and to obtain maximum validation scores in the established models. Here the steps;</p>
<p><strong>Exploratory Data Analysis:</strong>  The data set&#39;s structural data were checked. The types of variables in the dataset were examined. Size information of the dataset was accessed. The 0 values in the data set are missing values. Primarily these 0 values were replaced with NaN values. Descriptive statistics of the data set were examined.</p>
<p><strong>Data Preprocessing section;</strong> The NaN values missing observations were filled with the median values of whether each variable was diabetic or not. The outliers were determined by LOF and dropped. </p>
<p><strong>In model building;</strong> first, the base model was create and the test results were checked. Then categorical variables were edited and new features were added to the model.</p>
<p><strong>During Model Building;</strong> Logistic Regression, KNN, CART, Random Forests, GBM, XGBoost, LightGBM like using machine learning models Cross Validation Score were calculated. </p>
<p><strong>According to test results;</strong> GBM, XGBoost, LightGBM hyperparameter optimizations optimized to increase Cross Validation value.</p>
<p><strong>The model with the highest score after Hyper Parameter optimization was LGBM with 0.90 cross validation score</strong></p>



</div>